# Sonar - Decentralized Model Training Simulation (local)

DISCLAIMER: This is a proof-of-concept implementation. It does not represent a remotely product ready implementation or follow proper conventions for security, convenience, or scalability. It is part of a broader proof-of-concept demonstrating the vision of the OpenMined project, its major moving parts, and how they might work together.


# Getting Started: Installation

##### Step 1: install IPFS

- https://ipfs.io/docs/install/

##### Step 2: Turn on IPFS Daemon
Execute on command line:
> ipfs daemon

##### Step 3: Install Ethereum testrpc

- https://github.com/ethereumjs/testrpc

##### Step 4: Turn on testrpc with 1000 initialized accounts (each with some money)
Execute on command line:
> testrpc -a 1000

##### Step 5: install openmined/sonar and all dependencies (truffle)

##### Step 6: Locally Deploy Smart Contracts in openmined/sonar
From the OpenMined/Sonar repository root run
> truffle compile
> truffle migrate

you should see something like this when you run migrate:
```
Using network 'development'.

Running migration: 1_initial_migration.js
  Deploying Migrations...
  Migrations: 0xf06039885460a42dcc8db5b285bb925c55fbaeae
Saving successful migration to network...
Saving artifacts...
Running migration: 2_deploy_contracts.js
  Deploying ConvertLib...
  ConvertLib: 0x6cc86f0a80180a491f66687243376fde45459436
  Deploying ModelRepository...
  ModelRepository: 0xe26d32efe1c573c9f81d68aa823dcf5ff3356946
  Linking ConvertLib to MetaCoin
  Deploying MetaCoin...
  MetaCoin: 0x6d3692bb28afa0eb37d364c4a5278807801a95c5
```

The address after 'ModelRepository' is something you'll need to copy paste into the code
below when you initialize the "ModelRepository" object. In this case the address to be
copy pasted is `0xe26d32efe1c573c9f81d68aa823dcf5ff3356946`.

##### Step 7: execute the following code

# The Simulation: Diabetes Prediction

In this example, a diabetes research center (Cure Diabetes Inc) wants to train a model to try to predict the progression of diabetes based on several indicators. They have collected a small sample (42 patients) of data but it's not enough to train a model. So, they intend to offer up a bounty of $5,000 to the OpenMined commmunity to train a high quality model.

As it turns out, there are 400 diabetics in the network who are candidates for the model (are collecting the relevant fields). In this simulation, we're going to faciliate the training of Cure Diabetes Inc incentivizing these 400 anonymous contributors to train the model using the Ethereum blockchain.

Note, in this simulation we're only going to use the sonar and syft packages (and everything is going to be deployed locally on a test blockchain). Future simulations will incorporate mine and capsule for greater anonymity and automation.

### Imports and Convenience Functions

In [1]:
import warnings
import numpy as np
import phe as paillier
import time
from sonar.contracts_listclass import ModelRepository,Model,Gradient_List
from syft.he.paillier.keys import KeyPair,SecretKey,PublicKey
from syft.nn.linear import LinearClassifier
from sklearn.datasets import load_breast_cancer

start = time.time()

def get_balance(account):
    return repo.web3.fromWei(repo.web3.eth.getBalance(account),'ether')

warnings.filterwarnings('ignore')

### Setting up the Experiment

In [2]:
# for the purpose of the simulation, we're going to split our dataset up amongst
# the relevant simulated users

diabetes = load_breast_cancer()
y = diabetes.target
X = diabetes.data

validation = (X[46:51],y[46:51])
anonymous_diabetes_users = (X[52:],y[52:])

# we're also going to initialize the model trainer smart contract, which in the
# real world would already be on the blockchain (managing other contracts) before
# the simulation begins

# ATTENTION: copy paste the correct address (NOT THE DEFAULT SEEN HERE) from truffle migrate output.
repo = ModelRepository('0x130B4774b1010C4649057c802955B631d2114E53') # blockchain hosted model repository

No account submitted... using default[2]
Connected to ModelRepository:0x130B4774b1010C4649057c802955B631d2114E53


In [3]:


# we're going to set aside 10 accounts for our 42 patients
# Let's go ahead and pair each data point with each patient's 
# address so that we know we don't get them confused
patient_addresses = repo.web3.eth.accounts[1:15]
anonymous_diabetics = list(zip(patient_addresses,
                               anonymous_diabetes_users[0],
                               anonymous_diabetes_users[1]))

# we're going to set aside 1 account for Cure Diabetes Inc
cure_diabetes_inc = repo.web3.eth.accounts[1]
agg_addr = repo.web3.eth.accounts[2]

## Step 1: Cure Diabetes Inc Initializes a Model and Provides a Bounty

In [4]:
pubkey,prikey = KeyPair().generate(n_length=1024)
#pubkey,prikey=paillier.paillier.generate_paillier_keypair()
diabetes_classifier = LinearClassifier(desc="DiabetesClassifier",n_inputs=30,n_labels=2)
initial_error = diabetes_classifier.evaluate(validation[0],validation[1])
diabetes_classifier.encrypt(pubkey)
s1,s2=paillier.paillier.genKeyShares(prikey.sk,pubkey.pk)
st=SecretKey(s1)
sab=SecretKey(s2)
s3,s4=paillier.paillier.genKeyShares(s2,pubkey.pk)
sa=SecretKey(s3)
scb=SecretKey(s4)

diabetes_model = Model(owner=cure_diabetes_inc,
                       syft_obj = diabetes_classifier,
                       bounty = 2,
                       initial_error = initial_error,
                       target_error = 100,
                       best_error= initial_error
                      )
model_id = repo.submit_model(diabetes_model)
print('initial error',initial_error)

initial error 800


## Step 2: An Anonymous Patient Downloads the Model and Improves It

In [5]:
model_id

1

In [6]:
model = repo[model_id]

In [7]:
diabetic_address,input_data,target_data = anonymous_diabetics[0]

In [8]:
gradient=model.generate_gradient(diabetic_address,input_data,target_data)
model.submit_transformed_gradients(gradient,st)

## Step 3: Cure Diabetes Inc. Evaluates the Gradient 

In [9]:
#print (model.gradient_list)
#print(model.model_id)

In [10]:
old_balance = get_balance(diabetic_address)
print(old_balance)

77.9999999999952916


In [11]:
gradient_list=Gradient_List(model_id, repo=repo, model=model)
gradient_list[model_id]
model=repo[model_id]
avg_gradient=gradient_list.generate_gradient_avg(agg_addr,sa,alpha=2)
new_error,updatedModel = model.evaluate_gradient_from_avg(cure_diabetes_inc,agg_addr,avg_gradient,scb,prikey,pubkey,validation[0],validation[1])

len 1
type <class 'phe.paillier.EncryptedNumber'>


In [12]:
new_error

273221836002328

In [13]:
new_balance = get_balance(diabetic_address)
incentive = new_balance - old_balance
print(incentive)

-3.8565E-14


## Step 4: Rinse and Repeat

In [14]:
repo[model_id]

Desc:DiabetesClassifier
Owner:0xe7a6fe93c06be5a99551d32e1a0771b3da3aa4c3
Bounty:2
Initial Error:800
Best Error:800
Target Error:100
Model ID:1
Num Grads:1

In [15]:
for i,(addr, input, target) in enumerate(anonymous_diabetics):
        model = repo[model_id]
        print('model_id',model_id)
        
        # patient is doing this
        gradient=model.generate_gradient(addr,input,target)
        model.submit_transformed_gradients(gradient,st)
        
        # Cure Diabetes Inc does this
        old_balance = get_balance(agg_addr)
        print(old_balance)
        gradient_list=Gradient_List(model_id, repo=repo, model=model)
        gradient_list[model_id]
        avg_gradient=gradient_list.generate_gradient_avg(agg_addr,sa,alpha=2)
        new_error,updatedModel = model.evaluate_gradient_from_avg(cure_diabetes_inc,agg_addr,avg_gradient,scb,prikey,pubkey,validation[0],validation[1],alpha=2)
        print("new error from averaged gradients = "+str(new_error))
        print(model.best_error)
        incentive = (get_balance(agg_addr) - old_balance)
        print("incentive = "+str(incentive))
umodelid=repo.submit_updated_model(updatedModel)
model=repo.getUpdatedModel(umodelid)
end = time.time()
print('execution time', end - start)



model_id 1
102.000005174448574608
len 2
type <class 'phe.paillier.EncryptedNumber'>
new error from averaged gradients = 1092888823145511
800
incentive = 0.000001366110896550
model_id 1
102.000006540559338259
len 3
type <class 'phe.paillier.EncryptedNumber'>
new error from averaged gradients = 485728037145427
800
incentive = 6.07159914050E-7
model_id 1
102.000007147719252309
len 4
type <class 'phe.paillier.EncryptedNumber'>
new error from averaged gradients = 273221836002328
800
incentive = 3.41527162621E-7
model_id 1
102.00000748924641493
len 5
type <class 'phe.paillier.EncryptedNumber'>
new error from averaged gradients = 380756560763733
800
incentive = 4.75945568573E-7
model_id 1
102.000007965191983503
len 6
type <class 'phe.paillier.EncryptedNumber'>
new error from averaged gradients = 264414157057164
800
incentive = 3.30517563940E-7
model_id 1
102.000008295709547443
len 7
type <class 'phe.paillier.EncryptedNumber'>
new error from averaged gradients = 194263373245390
800
incentive =